In [55]:
# Import dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

# Import and read charity_data.csv
import pandas as pd 
application_df = pd.read_csv('../Resources/charity_data.csv')
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


# 3. Preprocess the data

In [56]:
# Drop non-beneficial ID columns
application_df = application_df.drop(columns=['EIN', 'NAME', 'STATUS', 'SPECIAL_CONSIDERATIONS'])

In [57]:
# APPLICATION_TYPE value counts for binning
application_values = application_df['APPLICATION_TYPE'].value_counts()

# Determine which values to replace if counts are less than 1000
replace_application = application_values[application_values < 1000].index.tolist()

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app, 'Other')

In [58]:
# CLASSIFICATION value counts for binning
classification_values = application_df['CLASSIFICATION'].value_counts()

# Determine which values to replace if counts are less than 1000
replace_class = classification_values[classification_values < 1000].index.tolist()

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls, 'Other')

In [59]:
# ASK_AMT bins
ask_values_0 = application_df['ASK_AMT'][application_df['ASK_AMT'] == 5000]
ask_values_1 = application_df['ASK_AMT'][(application_df['ASK_AMT'] > 5000)]

# Replace in dataframe
for amt in ask_values_0:
    application_df.ASK_AMT = application_df.ASK_AMT.replace(amt, '5000')
for amt in ask_values_1:
    application_df.ASK_AMT = application_df.ASK_AMT.replace(amt, '>5000')

In [60]:
# AFFILIATION value counts for binning
affiliation_values = application_df['AFFILIATION'].value_counts()

# Determine which values to replace if counts are less than 1000
replace_affiliation = affiliation_values[affiliation_values < 1000].index.tolist()

# Replace in dataframe
for aff in replace_affiliation:
    application_df.AFFILIATION = application_df.AFFILIATION.replace(aff, 'Other')

In [61]:
# ORGANIZATION value counts for binning
organization_values = application_df['ORGANIZATION'].value_counts()

# Determine which values to replace if counts are less than 1000
replace_organization = organization_values[organization_values < 1000].index.tolist()

# Replace in dataframe
for org in replace_organization:
    application_df.ORGANIZATION = application_df.ORGANIZATION.replace(org, 'Other')

In [62]:
# Generate the categorical variable list
application_cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()

In [63]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)

In [64]:
# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df, left_index=True, right_index=True)
application_df = application_df.drop(application_cat, 1)

In [65]:
# Split the preprocessed data into features and target arrays
y = application_df['IS_SUCCESSFUL'].values
X = application_df.drop(['IS_SUCCESSFUL'], 1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [66]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# 4. Design a neural network model

In [67]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 120
hidden_nodes_layer2 = 50
hidden_nodes_layer3 = 18

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="sigmoid"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation='relu'))

# Third hiZdden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 120)               4200      
_________________________________________________________________
dense_13 (Dense)             (None, 50)                6050      
_________________________________________________________________
dense_14 (Dense)             (None, 18)                918       
_________________________________________________________________
dense_15 (Dense)             (None, 1)                 19        
Total params: 11,187
Trainable params: 11,187
Non-trainable params: 0
_________________________________________________________________


In [68]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# 5. Create a callback

In [69]:
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs('optimized_checkpoints/', exist_ok=True)
checkpoint_path = 'optimized_checkpoints/weights.{epoch:02d}.hdf5'

# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=4000)

In [70]:
# Train the model
fit_model = nn.fit(X_train, y_train, epochs=200, callbacks=[cp_callback])

Epoch 1/200
804/804 [==============================] - 1s 894us/step - loss: 0.6130 - accuracy: 0.6814
Epoch 2/200
804/804 [==============================] - 1s 906us/step - loss: 0.5799 - accuracy: 0.7170
Epoch 3/200
804/804 [==============================] - 1s 991us/step - loss: 0.5731 - accuracy: 0.7212
Epoch 4/200
804/804 [==============================] - 1s 922us/step - loss: 0.5691 - accuracy: 0.7240
Epoch 5/200
752/804 [===========================>..] - ETA: 0s - loss: 0.5678 - accuracy: 0.7240
Epoch 00005: saving model to optimized_checkpoints/weights.05.hdf5
804/804 [==============================] - 1s 969us/step - loss: 0.5666 - accuracy: 0.7252
Epoch 6/200
804/804 [==============================] - 1s 960us/step - loss: 0.5644 - accuracy: 0.7272
Epoch 7/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5625 - accuracy: 0.7289
Epoch 8/200
804/804 [==============================] - 1s 971us/step - loss: 0.5603 - accuracy: 0.73020s - loss: 0.5598 - accu
Ep

804/804 [==============================] - 1s 1ms/step - loss: 0.5470 - accuracy: 0.7362
Epoch 64/200
804/804 [==============================] - 1s 962us/step - loss: 0.5468 - accuracy: 0.7349
Epoch 65/200
513/804 [==================>...........] - ETA: 0s - loss: 0.5438 - accuracy: 0.7368
Epoch 00065: saving model to optimized_checkpoints/weights.65.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5462 - accuracy: 0.7356
Epoch 66/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5462 - accuracy: 0.7359
Epoch 67/200
804/804 [==============================] - 1s 962us/step - loss: 0.5465 - accuracy: 0.7356
Epoch 68/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5462 - accuracy: 0.7346
Epoch 69/200
804/804 [==============================] - 1s 968us/step - loss: 0.5460 - accuracy: 0.7357
Epoch 70/200
499/804 [=================>............] - ETA: 0s - loss: 0.5413 - accuracy: 0.7404
Epoch 00070: saving model to optimized_chec

804/804 [==============================] - 1s 910us/step - loss: 0.5408 - accuracy: 0.7391
Epoch 125/200
265/804 [========>.....................] - ETA: 0s - loss: 0.5443 - accuracy: 0.7374
Epoch 00125: saving model to optimized_checkpoints/weights.125.hdf5
804/804 [==============================] - 1s 971us/step - loss: 0.5406 - accuracy: 0.7393
Epoch 126/200
804/804 [==============================] - 1s 923us/step - loss: 0.5408 - accuracy: 0.7390
Epoch 127/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5403 - accuracy: 0.7383: 0s - loss: 0.5403 - accuracy: 0.73
Epoch 128/200
804/804 [==============================] - 1s 908us/step - loss: 0.5406 - accuracy: 0.7395
Epoch 129/200
804/804 [==============================] - 1s 934us/step - loss: 0.5405 - accuracy: 0.73850s - loss: 0.5380 
Epoch 130/200
263/804 [========>.....................] - ETA: 0s - loss: 0.5416 - accuracy: 0.7417
Epoch 00130: saving model to optimized_checkpoints/weights.130.hdf5
804/804 [===

804/804 [==============================] - 1s 926us/step - loss: 0.5377 - accuracy: 0.7406
Epoch 185/200
 46/804 [>.............................] - ETA: 0s - loss: 0.5446 - accuracy: 0.7323
Epoch 00185: saving model to optimized_checkpoints/weights.185.hdf5
804/804 [==============================] - 1s 980us/step - loss: 0.5378 - accuracy: 0.7402
Epoch 186/200
804/804 [==============================] - 1s 904us/step - loss: 0.5373 - accuracy: 0.7398
Epoch 187/200
804/804 [==============================] - 1s 990us/step - loss: 0.5374 - accuracy: 0.7400
Epoch 188/200
804/804 [==============================] - 1s 922us/step - loss: 0.5373 - accuracy: 0.7407
Epoch 189/200
804/804 [==============================] - 1s 906us/step - loss: 0.5374 - accuracy: 0.7406
Epoch 190/200
  1/804 [..............................] - ETA: 0s - loss: 0.5212 - accuracy: 0.7500
Epoch 00190: saving model to optimized_checkpoints/weights.190.hdf5
804/804 [==============================] - 1s 942us/step - loss:

In [71]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f'Loss: {model_loss}, Accuracy: {model_accuracy}')

268/268 - 0s - loss: 1.0440 - accuracy: 0.6850
Loss: 1.0439964532852173, Accuracy: 0.6850146055221558


# 6. Export results

In [72]:
# Export the model to HDF5 file
nn.save('AlphabetSoupCharity_Optimization.h5')